In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = 'https://raw.githubusercontent.com/Dimassaputra5/DataQuest/refs/heads/main/training_dataset.csv'
data_validation = 'https://raw.githubusercontent.com/Dimassaputra5/DataQuest/refs/heads/main/validation_set.csv'
cc = pd.read_csv(data, 
                 encoding='utf-8',  # atau 'latin1', 'iso-8859-1', dll
                 sep=',')
cc_validation = pd.read_csv(data_validation, 
                 encoding='utf-8',  # atau 'latin1', 'iso-8859-1', dll
                 sep=',')
cc

In [ ]:
cc_validation

In [ ]:
cc.info()
cc_validation.info()

In [ ]:
# cc['status_perkawinan'].unique()
# cc['status_perkawinan'] = cc['status_perkawinan'].map({'menikah': 1, 'lajang':2, 'cerai': 3, 'unknown': -2})
# cc['status_perkawinan'].unique()
# ##########################################################
# cc_validation['status_perkawinan'] = cc_validation['status_perkawinan'].map({'menikah': 1, 'lajang':2, 'cerai': 3, 'unknown': -2})


In [ ]:
cc['RasioPekerjaSukuBunga'] = cc['jumlah_pekerja']/cc['suku_bunga_euribor_3bln']
###################################################
cc_validation['RasioPekerjaSukuBunga'] = cc_validation['jumlah_pekerja']/cc_validation['suku_bunga_euribor_3bln']

In [ ]:
cc['gagal_bayar_sebelumnya'].unique()
cc['gagal_bayar_sebelumnya'] = cc['gagal_bayar_sebelumnya'].map({'no': 0, 'yes': 1, 'unknown': -1})
cc['gagal_bayar_sebelumnya'].unique()
###############################################
cc_validation['gagal_bayar_sebelumnya'] = cc_validation['gagal_bayar_sebelumnya'].map({'no': 0, 'yes': 1, 'unknown': -1})


In [ ]:
cc['pinjaman_rumah'].unique()
cc['pinjaman_rumah'] = cc['pinjaman_rumah'].map({'no': 0, 'yes': 1, 'unknown': -1})
cc['pinjaman_rumah'].unique()
##############################################
cc_validation['pinjaman_rumah'] = cc_validation['pinjaman_rumah'].map({'no': 0, 'yes': 1, 'unknown': -1})


In [ ]:
cc['pinjaman_pribadi'].unique()
cc['pinjaman_pribadi'] = cc['pinjaman_pribadi'].map({'no': 0, 'yes': 1, 'unknown': -1})
cc['pinjaman_pribadi'].unique()
################################################
cc_validation['pinjaman_pribadi'] = cc_validation['pinjaman_pribadi'].map({'no': 0, 'yes': 1, 'unknown': -1})


In [ ]:
cc['jenis_kontak'].unique()
cc['jenis_kontak'] = cc['jenis_kontak'].map({'cellular': 0, 'telephone': 1})
##########################################################
cc_validation['jenis_kontak'] = cc_validation['jenis_kontak'].map({'cellular': 0, 'telephone': 1})
cc['jenis_kontak'].unique() 

In [ ]:
cc['bulan_kontak_terakhir'].unique()
cc['bulan_kontak_terakhir'] = cc['bulan_kontak_terakhir'].map({
    'jul': 7, 'nov': 11, 'may': 5, 'aug': 8, 'oct': 10, 'apr': 4, 'jun': 6, 'sep': 9, 'mar': 3, 'dec':12
}).astype(int)
cc['bulan_kontak_terakhir'].unique()
#####################################################################
cc_validation['bulan_kontak_terakhir'] = cc_validation['bulan_kontak_terakhir'].map({
    'jul': 7, 'nov': 11, 'may': 5, 'aug': 8, 'oct': 10, 'apr': 4, 'jun': 6, 'sep': 9, 'mar': 3, 'dec':12
}).astype(int)

In [ ]:
cc['hari_kontak_terakhir'].unique()
cc['hari_kontak_terakhir'] = cc['hari_kontak_terakhir'].map({
    'fri': 5, 'thu': 4, 'wed': 3, 'tue': 2, 'mon': 7})
cc['hari_kontak_terakhir'].unique()
##############################################################
cc_validation['hari_kontak_terakhir'] = cc_validation['hari_kontak_terakhir'].map({
    'fri': 5, 'thu': 4, 'wed': 3, 'tue': 2, 'mon': 7})

In [ ]:
cc['hasil_kampanye_sebelumnya'].unique()
cc['hasil_kampanye_sebelumnya'] = cc['hasil_kampanye_sebelumnya'].map({
    'nonexistent': -1, 'failure': 0, 'success': 1
})
###################################################################
cc_validation['hasil_kampanye_sebelumnya'] = cc_validation['hasil_kampanye_sebelumnya'].map({
    'nonexistent': -1, 'failure': 0, 'success': 1
})

In [ ]:
cc['berlangganan_deposito'] = cc['berlangganan_deposito'].astype(float)
cc.info()

In [ ]:
cc['total_kontak'] = cc['jumlah_kontak_kampanye_ini'] + cc['jumlah_kontak_sebelumnya']
###################################
cc_validation['total_kontak'] = cc_validation['jumlah_kontak_kampanye_ini'] + cc_validation['jumlah_kontak_sebelumnya']

In [ ]:
cc['memiliki_pinjaman'] = cc['pinjaman_pribadi'] | cc['pinjaman_rumah']
###################################
cc_validation['memiliki_pinjaman'] = cc_validation['pinjaman_pribadi'] | cc_validation['pinjaman_rumah']

In [ ]:
cc['rasio_ekonomi'] = cc['suku_bunga_euribor_3bln']/cc['indeks_harga_konsumen']
####################################
cc_validation['rasio_ekonomi'] = cc_validation['suku_bunga_euribor_3bln']/cc_validation['indeks_harga_konsumen']

In [ ]:
cc['variasi_pekerjaan_terhadap_suku_bunga'] = cc['tingkat_variasi_pekerjaan']/cc['suku_bunga_euribor_3bln']
#########################################################
cc_validation['variasipekerjaanterhadapsukubunga'] = cc_validation['tingkat_variasi_pekerjaan']/cc_validation['suku_bunga_euribor_3bln']


In [ ]:
cc.duplicated().sum()
cc.drop_duplicates(inplace=True)

In [ ]:
cc['rasio_jumlah_pekerja_variasi_pekerjaan'] = cc['jumlah_pekerja']/cc['tingkat_variasi_pekerjaan']
##############################################################
cc_validation['rasio_jumlah_pekerja_variasi_pekerjaan'] = cc_validation['jumlah_pekerja']/cc_validation['tingkat_variasi_pekerjaan']

In [ ]:
# cc['rasio_kampanye_sebelumnya_jumlah_kontak_sebelumnya'] = 

In [ ]:
# cc.drop(columns=['usia'])
# cc_validation.drop(columns=['usia'])

In [ ]:
cc_num = cc.select_dtypes(include=np.number)
corelation_matrix = cc_num.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corelation_matrix, annot=True ,cmap='coolwarm') 

In [ ]:
# # First create the correlation matrix
# correlation_matrix = cc_num.corr()

# # Convert the correlation matrix to a more manageable format
# # This will give us pairs of features and their correlation values
# correlations = []
# for i in range(len(correlation_matrix.columns)):
#     for j in range(i+1, len(correlation_matrix.columns)):  # Start from i+1 to avoid duplicates
#         col1 = correlation_matrix.columns[i]
#         col2 = correlation_matrix.columns[j]
#         corr_value = correlation_matrix.iloc[i, j]
#         if abs(corr_value) < 0.2:  # Check for absolute correlation > 0.4
#             correlations.append({
#                 'feature1': col1,
#                 'feature2': col2,
#                 'correlation': corr_value
#             })

# # Sort by absolute correlation value and print
# correlations.sort(key=lambda x: abs(x['correlation']), reverse=True)

# print("Strong correlations (|correlation| < 0.2):")
# print("-" * 50)
# for corr in correlations:
#     print(f"{corr['feature1']} vs {corr['feature2']}: {corr['correlation']:.3f}")

# # Alternatively, if you just want the values:
# # print("\nJust correlation values < 0.2:")
# # print("-" * 50)
# # for corr in correlations:
# #     print(f"{corr['correlation']:.3f}")

In [ ]:
cc_validation.info()
cc.info()

In [ ]:
cc['pendidikan'].unique()
cc['pendidikan'] = cc['pendidikan'].map({'TIDAK SEKOLAH': 0, 'Tidak Tamat SD': 1, 'SD': 2, 'SMP': 3, 'SMA': 4, 'Diploma': 5, 'Pendidikan Tinggi': 6, 'unknown': -2})
###############################################
cc_validation['pendidikan'] = cc_validation['pendidikan'].map({'TIDAK SEKOLAH': 0, 'Tidak Tamat SD': 1, 'SD': 2, 'SMP': 3, 'SMA': 4, 'Diploma': 5, 'Pendidikan Tinggi': 6, 'unknown': -2})


In [ ]:
cc.select_dtypes(include='object').info()

In [ ]:
cc = pd.get_dummies(cc, columns=['pekerjaan', 'status_perkawinan', 'pulau'], drop_first=True)
cc.info()
cc.head()
################################################
cc_validation = pd.get_dummies(cc_validation, columns=['pekerjaan', 'pulau'], drop_first=True)
cc_validation.info()
cc_validation.head()

# Modeling

In [ ]:
from cuml.ensemble import RandomForestClassifier
from cuml.preprocessing import StandardScaler
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score, roc_auc_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import optuna
import cudf
import cupy
import sqlite3

In [ ]:
storage_url = "sqlite:///optuna_study2.db"

In [ ]:
cc = cudf.DataFrame(cc)

In [ ]:
X = cc.drop(columns=['berlangganan_deposito'], axis=1)
y = cc['berlangganan_deposito']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 5, 300)
#     max_depth = trial.suggest_int('max_depth', 2, 40)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 40)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 40)
    
#     model = RandomForestClassifier(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             random_state=42,
#             bootstrap=True,
#             min_samples_split=min_samples_split,
#             min_samples_leaf=min_samples_leaf,
#             max_features='sqrt',
#             n_streams=1)

#     model.fit(X_train, y_train)

#     y_pred = model.predict_proba(X_val).to_numpy()[:, 1]
#     auc_score = roc_auc_score(y_val, y_pred)

    

#     return  auc_score

def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'lambda': trial.suggest_float('lambda', 0, 5),
        'alpha': trial.suggest_float('alpha', 0, 5),
    }

    model = xgb.XGBClassifier(**param, use_label_encoder=False)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    preds = model.predict_proba(X_val)
    auc_score = roc_auc_score(y_val, y_pred)
    return auc_score

study = optuna.create_study(direction='maximize', study_name='XGboost_Study',storage=storage_url, load_if_exists=True)
study.optimize(objective, n_trials = 100)

best_params = study.best_params
print("best parameter", best_params)

# best_model = RandomForestClassifier(
#             n_estimators=best_params['n_estimators'],
#             max_depth=best_params['max_depth'],
#             random_state=42,
#             bootstrap=True,
#             min_samples_split=best_params['min_samples_split'],
#             min_samples_leaf=best_params['min_samples_leaf'],
#             max_features='sqrt',
#             n_streams=1)

# best_model.fit(X_train, y_train)

best_model = xgb.XGBClassifier(**study.best_params, use_label_encoder=False)
best_model.fit(X_train, y_train)

y_pred = best_model.predict_proba(X_val).to_numpy()[:, 1]


auc_score = roc_auc_score(y_val, y_pred)
print(f"AUC Score: {auc_score:.4f}")

print(y_pred[:10]) 

In [ ]:
# X_train = cc.drop(columns=['berlangganan_deposito'], axis=1)
# y_train = cc['berlangganan_deposito']
# X_validation = cc_validation

# X_train = cudf.DataFrame(X_train)
# y_train = cudf.Series(y_train)
# X_validation = cudf.DataFrame(X_validation)

# scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_validation = scaler.transform(X_validation)

# model = RandomForestRegressor(
#         n_estimators=100,
#         max_depth=6,
#         random_state=42,
#         bootstrap=True,
#         min_samples_split=3,
#         max_features='sqrt',
#         n_streams=1)

# model.fit(X_train, y_train)

# y_val_pred = model.predict(X_validation)

# print(y_val_pred[:10]) 

# submission = cudf.DataFrame({'customer_number': cc_validation['customer_number'] ,'berlangganan_deposito': y_val_pred})
# submission.to_csv("Validation Result Submission.csv", index=False)